In [1]:
%load_ext bigdata

In [2]:
%pig_start

In [3]:
%timeout 300

In [4]:
## Se elimina el directorio de salida en el hdfs si existe
!hadoop fs -rm input/*
!hadoop fs -rm output/*
!hadoop fs -rm output2/*
!hadoop fs -rmdir input output
!hadoop fs -rmdir input output2

rm: `input/*': No such file or directory
rm: `output/*': No such file or directory
rm: `output2/*': No such file or directory
rmdir: `input': No such file or directory
rmdir: `output': No such file or directory
rmdir: `input': No such file or directory
rmdir: `output2': No such file or directory


In [5]:
!hadoop fs -put *.csv .
!hadoop fs -ls

put: `data.csv': File exists
Found 3 items
-rw-r--r--   1 root supergroup        615 2020-01-18 23:22 data.csv
-rw-r--r--   1 root supergroup       1741 2020-01-18 22:51 data.tsv
-rw-r--r--   1 root supergroup    2271958 2020-01-18 22:32 truck_event_text_partition.csv


In [6]:
!rm -rf input
!rm -rf output
!mkdir input
!mkdir output

In [7]:
%%writefile script.pig

-- crea la carpeta input in el HDFS
-- fs -mkdir input

-- copia los archivos del sistema local al HDFS
fs -put input/ .

-- carga de datos
--Punto 14

u = LOAD 'data.csv' USING PigStorage(',')
    AS (f1:INT, f2:CHARARRAY, f3:CHARARRAY, f4:CHARARRAY, f5:CHARARRAY, f6:INT);
    
y = FILTER u BY (NOT f5 matches '.*b.*');

b = FOREACH y GENERATE f5;

DUMP b;

-- escribe el archivo de salida
STORE b INTO 'output';

-- copia los warchivos del HDFS al sistema local
fs -get output/ .

Writing script.pig


In [8]:
!pig -execute 'run script.pig'

2020-01-18 23:56:44,652 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2020-01-18 23:56:48,354 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2020-01-18 23:56:48,691 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.reduce.markreset.buffer.percent is deprecated. Instead, use mapreduce.reduce.markreset.buffer.percent
2020-01-18 23:56:48,699 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.output.compress is deprecated. Instead, use mapreduce.output.fileoutputformat.compress
2020-01-18 23:56:48,727 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.submit.replication is deprecated. Instead, use mapreduce.client.submit.file.replication
2020-01-18 23:56:49,425 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker.http.address is deprecated. Instead, us

In [9]:
!hadoop fs -ls output/*

-rw-r--r--   1 root supergroup          0 2020-01-18 23:57 output/_SUCCESS
-rw-r--r--   1 root supergroup         78 2020-01-18 23:57 output/part-m-00000


In [10]:
!hadoop fs -cat output/part-m-00000

green
green
orange
viole
red
indigo
indigo
green
gray
red
orange
green
yellow


In [11]:
## Se elimina el directorio de salida en el hdfs si existe
!hadoop fs -rm input/*
!hadoop fs -rm output/*
!hadoop fs -rm output2/*
!hadoop fs -rmdir input output
!hadoop fs -rmdir input output2

rm: `input/*': No such file or directory
Deleted output/_SUCCESS
Deleted output/part-m-00000
rm: `output2/*': No such file or directory
rmdir: `input': No such file or directory
rmdir: `output2': No such file or directory


In [12]:
!rm -rf input
!rm -rf output